In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
try:
    from torchvision import transforms, utils
except:
    !conda install --yes torchvision --no-channel-priority
    from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
try:
    from torchvision import transforms, utils
except:
    !pip install torchvision
    from torchvision import transforms, utils
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50

In [3]:
ladiDataImport = pd.read_csv("http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_aggregated_responses_url.tsv",delimiter='\t',header='infer')
ladiDataImport.head(10)

,url,WorkerId,Answer
0,https://ladi.s3-us-west-2.amazonaws.com/Images...,0,['damage:flood/water']
1,https://ladi.s3-us-west-2.amazonaws.com/Images...,1,['damage:flood/water']
2,https://ladi.s3-us-west-2.amazonaws.com/Images...,2,['damage:flood/water']
3,https://ladi.s3-us-west-2.amazonaws.com/Images...,20,['damage:flood/water']
4,https://ladi.s3-us-west-2.amazonaws.com/Images...,0,['damage:flood/water']
5,https://ladi.s3-us-west-2.amazonaws.com/Images...,8,"['damage:flood/water', 'damage:washout']"
6,https://ladi.s3-us-west-2.amazonaws.com/Images...,9,"['damage:rubble', 'damage:flood/water']"
7,https://ladi.s3-us-west-2.amazonaws.com/Images...,0,['damage:rubble']
8,https://ladi.s3-us-west-2.amazonaws.com/Images...,1,['damage:none']
9,https://ladi.s3-us-west-2.amazonaws.com/Images...,11,['damage:flood/water']


In [4]:
# Strip off bracket and comma from the Answer catagory
ladiDataImport["Answer"] = ladiDataImport["Answer"].str.strip('[|]')
# split list of responses into multiple rows
ladiDataImport["Answer"] = ladiDataImport["Answer"].str.split(",",expand = True)
# remove the single quote character from either end of string
ladiDataImport["Answer"] = ladiDataImport["Answer"].str.strip('\'')
# add a column to help with aggregation when pivoting
ladiDataImport["response_count"] = 1
# Create a matrix with the number of workers who answered given label for given image
# using pivot table; filling in nan values with 0
label_matrix = ladiDataImport.pivot_table(values='response_count', 
                                          index='url', 
                                          columns='Answer', 
                                          aggfunc='sum',
                                          fill_value=0)
label_matrix.head(10)

Answer,damage:flood/water,damage:landslide,damage:misc,damage:none,damage:rubble,damage:smoke/fire,damage:washout,environment:dirt,environment:grass,environment:lava,environment:none,environment:rock,environment:sand,environment:shrubs,environment:snow/ice,environment:trees,infrastructure:bridge,infrastructure:building,infrastructure:communications-tower,infrastructure:dam-levee,infrastructure:none,infrastructure:pipe,infrastructure:railway,infrastructure:road,infrastructure:utility-line,infrastructure:water-tower,vehicle:aircraft,vehicle:boat,vehicle:car,vehicle:none,vehicle:truck,water:flooding,water:lake,water:none,water:ocean,water:puddle,water:river
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20145/DSC_0020_e34a1edc-6d5c-472e-847e-89dac3ed4519.jpg,2,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20145/DSC_0028_18dcd0d8-4b79-452e-9ade-604d4f13ddfd.jpg,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20145/DSC_0035_add5632e-eec2-42a4-a7db-8c42871164c2.jpg,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20145/DSC_0042_970b36fb-582b-4b51-a581-923efb394278.jpg,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20147/DSC_1575_787dc8f2-fb8f-4464-99a2-45ba5fc677c2.jpg,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20147/DSC_1579_0974d9a0-aa3a-4389-ba18-3c3dcd4a5e59.jpg,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,2,0,0,1,0,0,0,0,0,0,0,0,3,0,2,0,1,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20147/DSC_1583_3eedb6bb-da5a-4e50-bcbe-765338370ef7.jpg,3,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,2,0,1,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20147/DSC_1592_961cfdbc-0de8-4f06-9653-17c47cc7c464.jpg,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/1013/20147/DSC_1593_69721f5b-1689-4ee2-ae0b-6bdc8e62831d.jpg,3,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0,0,0,0,0


In [5]:
labels = ['infrastructure:bridge','infrastructure:building','infrastructure:dam-levee','infrastructure:none','infrastructure:railway','infrastructure:road','infrastructure:utility-line']
labelsWithoutNone = ['infrastructure:bridge','infrastructure:building','infrastructure:dam-levee','infrastructure:railway','infrastructure:road','infrastructure:utility-line']
#communication tower,no pipe,no water tower

def filterRows(row):

    arr = [row[x] for x in labels]
    return arr.count(0)

def proc(row): 
    arr = [row[x] for x in labelsWithoutNone]
    if row['infrastructure:none'] > max(arr):
#         print('true')
#         print(row)
        for x in labelsWithoutNone:
            row[x] = 0
        row['infrastructure:none'] = 1
    else:
#         print('false')
#         print(row)
        row['infrastructure:none'] = 0
        for x in labelsWithoutNone:
            if row[x] > 0: row[x] = 1
    return row
    

infrastructure_matrix = label_matrix[labels]
infrastructure_matrix = infrastructure_matrix[infrastructure_matrix.apply(filterRows,axis=1) != len(labels)]
infrastructure_matrix = infrastructure_matrix.apply(proc,axis =1)


# infrastructure_matrix = infrastructure_matrix[infrastructure_matrix.apply(filterRows,axis=1) != 6]
for i in labels:
    print("")
    print(i)
    print(infrastructure_matrix[infrastructure_matrix[i] != 0].count())
    print("")
# #1982 bridges
# #6294 buildings
# #65 comm towers 
# #137 dam-levees 
# #3325 none
# #101 pipe
# #108 railway
# #1609 road
# #192 utility line
# #75 water tower


infrastructure:bridge
Answer
infrastructure:bridge          1707
infrastructure:building        1707
infrastructure:dam-levee       1707
infrastructure:none            1707
infrastructure:railway         1707
infrastructure:road            1707
infrastructure:utility-line    1707
dtype: int64


infrastructure:building
Answer
infrastructure:bridge          5262
infrastructure:building        5262
infrastructure:dam-levee       5262
infrastructure:none            5262
infrastructure:railway         5262
infrastructure:road            5262
infrastructure:utility-line    5262
dtype: int64


infrastructure:dam-levee
Answer
infrastructure:bridge          113
infrastructure:building        113
infrastructure:dam-levee       113
infrastructure:none            113
infrastructure:railway         113
infrastructure:road            113
infrastructure:utility-line    113
dtype: int64


infrastructure:none
Answer
infrastructure:bridge          3328
infrastructure:building        3328
infrastructure

In [6]:
sample_size=80
samples = infrastructure_matrix[infrastructure_matrix['infrastructure:bridge'] == 1].sample(sample_size)
for col in infrastructure_matrix.columns[1:]:
    samples = samples.append(infrastructure_matrix[infrastructure_matrix[col] == 1].sample(sample_size))
samples = samples.reset_index().drop_duplicates(subset='url', keep='first').set_index('url')
samples.head(10)

Answer,infrastructure:bridge,infrastructure:building,infrastructure:dam-levee,infrastructure:none,infrastructure:railway,infrastructure:road,infrastructure:utility-line
url,,,,,,,
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/5/211119/DSC_0118_4cca5d88-f043-4ece-84ad-a6dbac834d1e.jpg,1,0,1,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/9157/616220/DSC_8324_927c82a2-ee7d-45ff-b10a-b46308c53ce2.jpg,1,1,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/9075/614004/A0019058_e738e8d5-a56a-4569-89c2-630222abfc10.jpg,1,1,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/5/615474/Dec17201_bbe01258-8f4c-4000-a0f3-7bdb7bfdfb1f.jpg,1,0,0,0,0,0,1
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/9096/615127/TimCooli_2b90d462-6c61-44bb-83a2-86bfec3bd736.jpg,1,1,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/9075/614088/A0074_V__7d391992-0887-4983-ba25-4b82d680baa2.jpg,1,1,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/5/615428/VIRB0086_7e3c2280-0c47-4a1d-aacb-c5c1320e560f.jpg,1,0,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/9075/614023/A0069AP-_c3540537-0588-4405-a066-9b0d8ede8c11.jpg,1,1,0,0,0,0,0
https://ladi.s3-us-west-2.amazonaws.com/Images/FEMA_CAP/9096/615129/BrettGro_5e430c1b-9b9c-463b-a90e-0d3993d91316.jpg,1,1,0,0,0,0,0


In [7]:
metadata_csv = 'infrastructure_sample_metadata.csv'
label_csv = 'infrastructure_sample_label.csv'

In [8]:
# Load ladi_images_metadata.csv
metadata = pd.read_csv('http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_images_metadata.csv')

# sampling
# sample_size=1000
# flood_sample = flood_examples.sample(sample_size)
# non_flood_sample = non_flood_examples.sample(sample_size)

# creating a df with True/False labels for flooding
# training_flood = pd.DataFrame(index=flood_sample.index, data={'label':True}).reset_index()
# training_non_flood = pd.DataFrame(index=non_flood_sample.index, data={'label':False}).reset_index()
# label_df = pd.concat([training_flood, training_non_flood], ignore_index=True)

samples.to_csv(label_csv)

# create list of urls to download
samples.reset_index()['url'].to_csv('urls_to_download.csv', index=False, header=False)

#get metada
infrastructure_metadata = metadata[metadata['url'].isin(samples.index)]
# flood_metadata = metadata[metadata['url'].isin(flood_sample.index)]
# not_flood_metadata = metadata[metadata['url'].isin(non_flood_sample.index)]
# training_metadata = pd.concat([flood_metadata, not_flood_metadata], ignore_index=True)

infrastructure_metadata.to_csv(metadata_csv)

In [9]:
# !mkdir -p training_images
# !wget --content-disposition --trust-server-names -i urls_to_download.csv -P training_images/

In [10]:
scale = transforms.Resize(768)
crop = transforms.RandomCrop(512)
rotate = transforms.RandomRotation(20)
flip_demo = transforms.RandomHorizontalFlip(1) # flip with 100% chance just to demo
flip = transforms.RandomHorizontalFlip(p=0.5)
composed = transforms.Compose([scale,
                               crop,
                               rotate,
                               flip_demo])

In [11]:
# convenient function for showing the images
def show_image(image):
    plt.imshow(image)
    # pause a bit so that plots are updated
    plt.pause(0.01)

def convert_url_to_local_path(url):
    '''
    gets the location of the downloaded image
    '''
    return 'training_images/'+url.split('/')[-1]

class InfrastructureSampleDataset(Dataset):

    def __init__(self, metadata_csv, label_csv, transform = None):
        """
        Args:
            metadata_csv (string): Path to the csv file with metadata.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.infrastructure_sample_metadata = pd.read_csv(metadata_csv)
        # get the path in the shared directory
        self.infrastructure_sample_metadata['local_path'] = self.infrastructure_sample_metadata['url'].apply(convert_url_to_local_path)
        self.infrastructure_sample_label = pd.read_csv(label_csv)
        self.infrastructure_sample_data = pd.merge(self.infrastructure_sample_metadata, 
                                        self.infrastructure_sample_label,
                                       on="url")
        self.transform = transform
        
    def __len__(self):
        return len(self.infrastructure_sample_metadata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ## Load images from local directory. There is no need to redownload images to local machine. ##
        local_path = self.infrastructure_sample_metadata.iloc[idx]['local_path']
        url = self.infrastructure_sample_metadata.iloc[idx]['url']
        try:
            image = Image.fromarray(io.imread(local_path))
            img_name = local_path
        except:
            image = Image.fromarray(io.imread(url))
            img_name = url
        uuid = self.infrastructure_sample_data.iloc[idx, 1]
        timestamp = self.infrastructure_sample_data.iloc[idx, 2]
        gps_lat = self.infrastructure_sample_data.iloc[idx, 3]
        gps_lon = self.infrastructure_sample_data.iloc[idx, 4]
        gps_alt = self.infrastructure_sample_data.iloc[idx, 5]
        file_size = self.infrastructure_sample_data.iloc[idx, 6]
        width = self.infrastructure_sample_data.iloc[idx, 7]
        height = self.infrastructure_sample_data.iloc[idx, 8]
        label = torch.tensor(self.infrastructure_sample_data.iloc[idx, -7:].tolist(), dtype=torch.float)
        
        if self.transform:
            image = self.transform(image)

        sample = {'image': image, 
                  'image_name': img_name, 
                  'infrastructure_labels': label, 
                  'uuid': uuid, 
                  'timestamp': timestamp, 
                  'gps_lat': gps_lat, 
                  'gps_lon': gps_lon, 
                  'gps_alt': gps_alt, 
                  'orig_file_size': file_size, 
                  'orig_width': width, 
                  'orig_height': height}

        return sample
    def __str__(self):
        return self.infrastructure_sample_data.to_string()
    
    def dataset(self):
        return self.infrastructure_sample_data
    def dataset_type(self):
        return type(self.infrastructure_sample_data)
    def get_columns(self):
        return self.infrastructure_sample_data.columns

In [12]:
infrastructure_sample_dataset = InfrastructureSampleDataset(metadata_csv = metadata_csv, label_csv = label_csv)

In [13]:
transformed_dataset = InfrastructureSampleDataset(metadata_csv = metadata_csv, 
                                       label_csv = label_csv, 
                                       transform = transforms.Compose([scale, 
                                                                       crop, 
                                                                       rotate, 
                                                                       flip, 
                                                                       transforms.ToTensor()]
                                                                     )
                                      )

In [14]:
# dataloader = DataLoader(transformed_dataset, batch_size=4, shuffle=True, num_workers=4)

In [15]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

batch_size = 16
test_split_ratio = .1
shuffle_dataset = True
random_seed = 42
# num_workers = 1

# Creating data indices for training and validation splits:
dataset_size = len(transformed_dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split_ratio * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(transformed_dataset, batch_size=batch_size,
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(transformed_dataset, batch_size=batch_size,
                                                sampler=test_sampler)

In [16]:
transformed_dataset[345]['image'].shape

torch.Size([3, 512, 512])

In [17]:
len(train_indices)

489

In [18]:
# dataiter = iter(train_loader)
# images, labels = dataiter.next()
# print(type(images))
# print(images.shape)
# print(labels.shape)
# for i, data in enumerate(train_loader, 0):

In [19]:
import torch.nn as nn
import torch.nn.functional as F
try:
    from cnn_finetune import make_model
except:
    !pip install cnn-finetune
    from cnn_finetune import make_model

In [20]:
net = make_model('resnet50', num_classes=7, pretrained=True).cuda()

In [27]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
def get_checkpoint_path(epoch):
    return f'epoch_checkpoints/flood_checkpoint_epoch{epoch}.pth'

torch.backends.cudnn.benchmark = True # flag for some GPU optimizations
starting_epoch = 1
additional_epochs = 5
if starting_epoch > 1:
    net.load_state_dict(torch.load(get_checkpoint_path(starting_epoch)))
for epoch in range(starting_epoch, starting_epoch+additional_epochs):  # loop over the dataset multiple times
    running_acc, running_loss = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
#         print(data['image'].shape)
#         print(data['damage_labels'])
#         print(data['damage_labels'].shape)
        inputs = data['image'].cuda()
        
#         labels = zip(data['damage:flood/water'], data['damage:rubble'], data['damage:smoke/fire'], data['damage:none']).cuda()
#         labels = data[['damage:flood/water','damage:rubble','damage:smoke/fire','damage:none']].cuda()
        labels = data['infrastructure_labels'].cuda()
        
#         labels = t.cuda()
        # casting int to long for loss calculation#
#         labels = labels.float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
#         outputs = torch.sigmoid(net(inputs))
        outputs = net(inputs)
        print(torch.sigmoid(outputs))
        print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        pred = (outputs > 0.5).float()
        correct = (pred == labels).all(1).float().sum()
        running_acc += correct
        if (i+1) % 10 == 0:    # print every 10 mini-batches         
            print(f'[Epoch {epoch}/{additional_epochs}, Batch {i+1}] Loss: {running_loss/10} Accuracy: {correct/(len(labels)*10)}')
            running_loss, running_acc = 0.0, 0.0
    # save the model
    PATH = get_checkpoint_path(epoch)
    torch.save(net.state_dict(), PATH)
print('Finished Training')

tensor([[0.1471, 0.7125, 0.1231, 0.2021, 0.2127, 0.1549, 0.1897],
        [0.0945, 0.1765, 0.1504, 0.3945, 0.1153, 0.2899, 0.1551],
        [0.3713, 0.9459, 0.0342, 0.0410, 0.1530, 0.1289, 0.2156],
        [0.1219, 0.2447, 0.1483, 0.1190, 0.1300, 0.6419, 0.2121],
        [0.7227, 0.8802, 0.0579, 0.0439, 0.0644, 0.2998, 0.1083],
        [0.3226, 0.8796, 0.0691, 0.0940, 0.1225, 0.1348, 0.1469],
        [0.2626, 0.4042, 0.0988, 0.1643, 0.1394, 0.3656, 0.1683],
        [0.1038, 0.2964, 0.2515, 0.4546, 0.0673, 0.1613, 0.2100],
        [0.0788, 0.2232, 0.1097, 0.2905, 0.1388, 0.4527, 0.1888],
        [0.4467, 0.8764, 0.0710, 0.0676, 0.0705, 0.1700, 0.1456],
        [0.3104, 0.4103, 0.4467, 0.2014, 0.0832, 0.2982, 0.2062],
        [0.4570, 0.6690, 0.2105, 0.0737, 0.1417, 0.2019, 0.0981],
        [0.7782, 0.7033, 0.1630, 0.0355, 0.1656, 0.1385, 0.0481],
        [0.0964, 0.6428, 0.0710, 0.1416, 0.2589, 0.3955, 0.1281],
        [0.1530, 0.3912, 0.1336, 0.1151, 0.2784, 0.6135, 0.1483],
        [0

In [ ]:
del inputs
del labels
del outputs
del loss
del PATH

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
inputs.shape

In [ ]:
labels.shape

In [ ]:
print(data['damage_labels'])

In [ ]:
print(data['image'].shape)
print(data['damage_labels'].shape)

In [ ]:
print(outputs.shape)

In [ ]:
print(loss.shape)

In [ ]:
criterion(outputs, labels)

In [ ]:
data['image'][10][0][1]

In [ ]:
torch.tensor(data['damage_labels'])

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(test_loader)
single_iter = dataiter.next()
images = single_iter['image']
labels = single_iter['damage:flood/water']

In [ ]:
net.load_state_dict(torch.load(PATH))

outputs = net(images.cuda())
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % predicted[j].cpu()
                              for j in range(batch_size)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images = data['image'].cuda()
        labels = data['damage:flood/water'].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
truth_labels = []
predicted_labels = []
with torch.no_grad():
    for data in test_loader:
        images = data['image'].cuda()
        labels = data['damage:flood/water'].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        truth_labels.append(labels.cpu())
        predicted_labels.append(predicted.cpu())
truth_labels = np.concatenate([x.numpy() for x in truth_labels])
predicted_labels = np.concatenate([x.numpy() for x in predicted_labels])

In [ ]:
import sklearn.metrics
confusion_matrix = sklearn.metrics.confusion_matrix(truth_labels, predicted_labels)
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix, ['flood','no flood'])
disp.plot()